In [ ]:
event_hub_namespace = "iesstsabbdbaa-grp-06-10"
topic1_consumer_string = 'Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=consumer;SharedAccessKey=5RErxrMbiKdB6RKF8OrbXS8MJxM8gwb5D+AEhIB3zAA=;EntityPath=passenger_requests_10'
topic2_consumer_string = 'Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=consumer;SharedAccessKey=AP+NyIt9D0Jg/yC9+ATTtW1/03WuNvUB4+AEhAkSlUE=;EntityPath=ride_status_10'
topic1_name = "passenger_requests_10"
topic2_name = "ride_status_10"
consumer_String = "Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=Consumer_10;SharedAccessKey=jPZjSqQc7HIox8LooseofiUY2mg5/pO/J+AEhDJ4b50="

# Azure Blob Storage
account_name = "iesstsabbdbaa"
account_key = 'ZT6z+TYSxF0Xdm0vOCRbIpWoBss2BxOU0EcP2UDceddHX7Kyi8gyJvjyWG5THNp2HOprCHmblb2f+AStp8mAGw=='
container_name = "group10"

In [ ]:
import os, time, subprocess

spark_version = subprocess.run(
    "curl -s https://downloads.apache.org/spark/ | grep -o 'spark-3\\.[0-9]\\+\\.[0-9]\\+' | sort -V | tail -1",
    shell=True, capture_output=True, text=True
).stdout.strip()

spark_release=spark_version
hadoop_version='hadoop3'

start=time.time()
os.environ['SPARK_RELEASE']=spark_release
os.environ['HADOOP_VERSION']=hadoop_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_release}-bin-{hadoop_version}"

In [ ]:
if True:
  # Run below commands in google colab
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null # install Java8
  !wget -q http://apache.osuosl.org/spark/${SPARK_RELEASE}/${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # download spark-3.3.X
  !tar xf ${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # unzip it

  !pip install -q findspark # install findspark
  # findspark find your Spark Distribution and sets necessary environment variables

In [ ]:
import findspark
findspark.init()

# Check the pyspark version
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col, from_unixtime, to_timestamp, window, session_window
import pyspark.sql.functions as F

In [ ]:
import findspark
findspark.init()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.5.5


In [ ]:
jar_dependencies = ",".join([
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5",  # Updated to 3.5.5
    "org.apache.spark:spark-avro_2.12:3.5.5",            # Updated to 3.5.5
    "org.apache.hadoop:hadoop-azure:3.3.1",              # Hadoop Azure connector (unchanged)
    "com.microsoft.azure:azure-storage:8.6.6"            # Azure Blob SDK (unchanged)
])

In [ ]:
spark = SparkSession \
    .builder \
    .appName("StreamingAVROFromKafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config("spark.jars.packages", jar_dependencies) \
    .config(f"fs.azure.account.key.{account_name}.blob.core.windows.net", account_key) \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
kafkaConf1 = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{topic1_consumer_string}";',
    "subscribe": topic1_name,
    "startingOffsets": "latest",
    "enable.auto.commit": "true ",
    "groupIdPrefix": "Stream_Analytics_",
    "auto.commit.interval.ms": "5000"
}

kafkaConf2 = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{topic2_consumer_string}";',
    "subscribe": topic2_name,
    "startingOffsets": "latest",
    "enable.auto.commit": "true ",
    "groupIdPrefix": "Stream_Analytics_",
    "auto.commit.interval.ms": "5000"
}

In [ ]:
PASSENGER_REQUEST_SCHEMA = """
{
  "type": "record",
  "name": "PassengerRequest",
  "fields": [
    {"name": "request_id", "type": "string", "doc": "Unique identifier for the passenger request."},
    {"name": "user_id", "type": "string", "doc": "Unique identifier for the user making the request."},
    {"name": "event_type", "type": {"type": "enum", "name": "EventType", "symbols": ["request", "cancel"]}, "doc": "Type of event, either a ride request or a cancellation."},
    {"name": "timestamp", "type": "long", "doc": "Epoch timestamp indicating when the request was made."},
    {"name": "pickup_latitude", "type": "double", "doc": "Latitude coordinate of the pickup location."},
    {"name": "pickup_longitude", "type": "double", "doc": "Longitude coordinate of the pickup location."},
    {"name": "pickup_address", "type": "string", "doc": "Readable address of the pickup location."},
    {"name": "dropoff_latitude", "type": "double", "doc": "Latitude coordinate of the dropoff location."},
    {"name": "dropoff_longitude", "type": "double", "doc": "Longitude coordinate of the dropoff location."},
    {"name": "dropoff_address", "type": "string", "doc": "Readable address of the dropoff location."},
    {"name": "payment_method", "type": {"type": "enum", "name": "PaymentMethod", "symbols": ["credit_card", "debit_card", "cash", "mobile_payment"]}, "doc": "Payment method chosen by the user."},
    {"name": "vehicle_type", "type": {"type": "enum", "name": "VehicleType", "symbols": ["standard", "premium", "luxury", "pool"]}, "doc": "Type of vehicle requested for the ride."},
    {"name": "num_passengers", "type": "int", "doc": "Number of passengers included in the ride request."},
    {"name": "fare_estimate", "type": "float", "doc": "Estimated fare for the requested ride."},
    {"name": "ride_purpose", "type": {"type": "enum", "name": "RidePurpose", "symbols": ["commute", "airport", "business", "leisure"]}, "doc": "Purpose of the ride request."},
    {"name": "device_type", "type": {"type": "enum", "name": "DeviceType", "symbols": ["mobile", "web"]}, "doc": "Type of device used to make the ride request."}]
}
"""

RIDE_STATUS_SCHEMA = """
{
  "type": "record",
  "name": "RideStatus",
  "fields": [
    {"name": "ride_id", "type": "string", "doc": "Unique identifier for the ride."},
    {"name": "request_id", "type": "string", "doc": "Identifier linking this ride status to the original ride request."},
    {"name": "status", "type": {"type": "enum", "name": "RideStatusEnum", "symbols": ["accepted", "ongoing", "completed", "cancelled"]}, "doc": "Current status of the ride."},
    {"name": "timestamp", "type": "long", "doc": "Epoch timestamp indicating when the status was recorded."},
    {"name": "driver_id", "type": "string", "doc": "Unique identifier for the driver assigned to the ride."},
    {"name": "estimated_arrival_time", "type": "long", "doc": "Estimated epoch timestamp of when the driver is expected to arrive at the pickup location."},
    {"name": "actual_arrival_time", "type": ["null", "long"], "default": "null", "doc": "Actual epoch timestamp when the driver arrived at the pickup location. Null if not yet arrived."},
    {"name": "ride_duration", "type": ["null", "int"], "default": "null", "doc": "Duration of the ride in minutes. Null if ride is not completed."},
    {"name": "distance_traveled", "type": ["null", "float"], "default": "null", "doc": "Total distance traveled during the ride in kilometers. Null if ride is not completed."},
    {"name": "fare", "type": "float", "doc": "Final fare amount for the ride."},
    {"name": "surge_multiplier", "type": "float", "doc": "Surge pricing multiplier applied to the fare."},
    {"name": "traffic_condition", "type": {"type": "enum", "name": "TrafficCondition", "symbols": ["low", "medium", "high"]}, "doc": "Traffic conditions during the ride."},
    {"name": "weather_condition", "type": {"type": "enum", "name": "WeatherCondition", "symbols": ["clear", "rainy", "snowy", "foggy", "stormy"]}, "doc": "Weather conditions at the time of the ride."},
    {"name": "day_of_week", "type": {"type": "enum", "name": "DayOfWeek", "symbols": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, "doc": "Day of the week when the ride took place."},
    {"name": "driver_rating", "type": ["null", "float"], "default": "null", "doc": "Driver's rating for the ride (scale 1-5). Null if ride not completed."},
    {"name": "cancellation_reason", "type": ["null", {"type": "enum", "name": "CancellationReason", "symbols": ["user_request", "driver_unavailable", "technical_issue"]}], "default": "null", "doc": "Reason for ride cancellation, if applicable. Null if not cancelled."},
    {"name": "payment_status", "type": {"type": "enum", "name": "PaymentStatus", "symbols": ["pending", "completed", "failed"]}, "doc": "Current status of the ride payment."}]
}
"""

In [ ]:
# Read from Event Hub using Kafka for topic1
df1 = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaConf1)

# Read from Event Hub using Kafka for topic2
df2 = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaConf2)

In [ ]:
# Deserialize the AVRO messages from the value column
df1 = df1.load()
df1 = df1.select(from_avro(df1.value, PASSENGER_REQUEST_SCHEMA).alias("passenger_request"))

# Deserialize the AVRO messages from the value column
df2 = df2.load()
df2 = df2.select(from_avro(df2.value, RIDE_STATUS_SCHEMA).alias("ride_status"))

In [ ]:
from pyspark.sql.functions import col

df1 = df1.select(
    col("passenger_request.request_id"),
    col("passenger_request.user_id"),
    col("passenger_request.event_type"),
    col("passenger_request.timestamp"),
    col("passenger_request.pickup_latitude"),
    col("passenger_request.pickup_longitude"),
    col("passenger_request.pickup_address"),
    col("passenger_request.dropoff_latitude"),
    col("passenger_request.dropoff_longitude"),
    col("passenger_request.dropoff_address"),
    col("passenger_request.payment_method"),
    col("passenger_request.vehicle_type"),
    col("passenger_request.num_passengers"),
    col("passenger_request.fare_estimate"),
    col("passenger_request.ride_purpose"),
    col("passenger_request.device_type")
)

df2 = df2.select(
    col("ride_status.ride_id"),
    col("ride_status.request_id"),
    col("ride_status.status"),
    col("ride_status.timestamp"),
    col("ride_status.driver_id"),
    col("ride_status.estimated_arrival_time"),
    col("ride_status.actual_arrival_time"),
    col("ride_status.ride_duration"),
    col("ride_status.distance_traveled"),
    col("ride_status.fare"),
    col("ride_status.surge_multiplier"),
    col("ride_status.traffic_condition"),
    col("ride_status.weather_condition"),
    col("ride_status.day_of_week"),
    col("ride_status.driver_rating"),
    col("ride_status.cancellation_reason"),
    col("ride_status.payment_status")
)

In [ ]:
df1.printSchema()

root
 |-- request_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_address: string (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_address: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- num_passengers: integer (nullable = true)
 |-- fare_estimate: float (nullable = true)
 |-- ride_purpose: string (nullable = true)
 |-- device_type: string (nullable = true)



In [ ]:
df2.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- request_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- driver_id: string (nullable = true)
 |-- estimated_arrival_time: long (nullable = true)
 |-- actual_arrival_time: long (nullable = true)
 |-- ride_duration: integer (nullable = true)
 |-- distance_traveled: float (nullable = true)
 |-- fare: float (nullable = true)
 |-- surge_multiplier: float (nullable = true)
 |-- traffic_condition: string (nullable = true)
 |-- weather_condition: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- driver_rating: float (nullable = true)
 |-- cancellation_reason: string (nullable = true)
 |-- payment_status: string (nullable = true)



In [ ]:
output_path = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/stream-output/"
checkpoint_path = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/checkpoint/"

output_path, checkpoint_path

('wasbs://group10@iesstsabbdbaa.blob.core.windows.net/stream-output/',
 'wasbs://group10@iesstsabbdbaa.blob.core.windows.net/checkpoint/')

In [ ]:
!pip install azure-identity azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
df1.writeStream \
    .format("console") \
    .start() \
    .awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = ade37c7d-e0d3-4736-914e-ccefd125d7cd, runId = e093fe4e-da15-4d27-ab7a-4cbeaa32d7a4] terminated with exception: Failed to create new KafkaAdminClient

In [ ]:
query_passenger = df1.writeStream \
    .format("parquet") \
    .option("checkpointLocation", f"{checkpoint_path}/passenger") \
    .option("path", f"{output_path}/passenger") \
    .queryName("passenger_query") \
    .trigger(processingTime="5 seconds") \
    .start()

q = spark.streams.get("passenger_query")
print(q.lastProgress)        # estado del último micro-batch
print(q.recentProgress)      # lista de batches más recientes

query_passenger.awaitTermination()

query_ride = df2.writeStream \
    .format("parquet") \
    .option("checkpointLocation", f"{checkpoint_path}/ride") \
    .option("path", f"{output_path}/ride") \
    .queryName("ride_query") \
    .trigger(processingTime="5 seconds") \
    .start()

query_ride.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.5.5-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.5.5-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Get the list of active streaming queries
active_queries = spark.streams.active

# Print details about each active query
for query in active_queries:
    print(f"Query Name: {query.name}")
    print(f"Query ID: {query.id}")
    print(f"Query Status: {query.status}")
    print(f"Is Query Active: {query.isActive}")
    print("-" * 50)